## Monk Library - https://github.com/Tessellate-Imaging/monk_v1

## Monk is an opensource low-code tool for computer vision and deep learning

Monk features

    low-code
    unified wrapper over major deep learning framework - keras, pytorch, gluoncv
    syntax invariant wrapper

Enables
  - to create, manage and version control deep learning experiments
  - to compare experiments across training metrics
  - to quickly find best hyper-parameters

At present it only supports transfer learning, but we are working each day to incorporate
  - GUI based custom model creation
  - various object detection and segmentation algorithms
  - deployment pipelines to cloud and local platforms
  - acceleration libraries such as TensorRT
  - preprocessing and post processing libraries
    
    
### Feature showcase
  - Custom Model Creation with Monk in Gluon Backend (Soon will be released for pytorch and keras backends too)


To contribute to Monk AI or Pytorch RoadMap repository raise an issue in the git-repo or dm us on linkedin
  - Abhishek - https://www.linkedin.com/in/abhishek-kumar-annamraju/
  - Akash - https://www.linkedin.com/in/akashdeepsingh01/



In [ ]:
# Since Monk supports Image as inputs

In [ ]:
# Convert CSV data to images
import pandas as pd
import numpy as np
import cv2
import os
from tqdm import tqdm_notebook as tqdm

os.mkdir("images");
os.mkdir("images/train");
os.mkdir("images/test");

In [ ]:
#Training data
dv = pd.read_csv("/kaggle/input/Kannada-MNIST/train.csv");
for i in tqdm(range(60000)):
    x = dv.iloc[i].to_numpy();
    label = str(x[0])
    img = x[1:]
    img = np.reshape(img, (28, 28));
    img = np.expand_dims(img, axis=0);
    img = np.vstack((img, img, img))
    img = np.swapaxes(img, 0, 1);
    img = np.swapaxes(img, 1, 2);
    if(not os.path.isdir("images/train/" + label)):
        os.mkdir("images/train/" + label);
    
    cv2.imwrite("images/train/" + label + "/" + str(i) + ".jpg", img);
    
    
#Testing data
dv = pd.read_csv("/kaggle/input/Kannada-MNIST/test.csv");
for i in tqdm(range(5000)):
    x = dv.iloc[i].to_numpy();
    id_ = str(x[0])
    img = x[1:]
    img = np.reshape(img, (28, 28));
    img = np.expand_dims(img, axis=0);
    img = np.vstack((img, img, img))
    img = np.swapaxes(img, 0, 1);
    img = np.swapaxes(img, 1, 2);
    cv2.imwrite("images/test/" + str(id_) + ".jpg", img);

In [ ]:
# Since the kernel cannot use internet, the installation got a little lengthy

In [ ]:
import os
os.listdir("/kaggle/input/monk-kaggle/kaggle/")

In [ ]:
import os
os.listdir("/kaggle/input/monk-kaggle/kaggle/installs")

In [ ]:
!cd /kaggle/input/monk-kaggle/kaggle/installs/ && pip install PyLg-1.3.3-py3-none-any.whl

In [ ]:
!cd /kaggle/input/monk-kaggle/kaggle/installs/ && pip install blessings-1.7-py3-none-any.whl

In [ ]:
!cd /kaggle/input/monk-kaggle/kaggle/installs/ && pip install netron-3.7.3-py2.py3-none-any.whl

In [ ]:
!cd /kaggle/input/monk-kaggle/kaggle/installs/ && pip install gpustat-0.6.0-py3-none-any.whl

In [ ]:
import sys
sys.path.append("/kaggle/input/monk-kaggle/kaggle/monk_v1/monk/")
sys.path.append("/kaggle/input/monk-kaggle/kaggle/installs/")

In [ ]:
from gluon_prototype import prototype

In [ ]:
gtf = prototype(verbose=1);
gtf.Prototype("sample-project", "sample-experiment-1");

In [ ]:
gtf.Dataset_Params(dataset_path="images/train/", 
                   split=0.8, input_size=28, 
                batch_size=16, shuffle_data=True, num_processors=3);

# Transform
gtf.apply_random_horizontal_flip(train=True, val=True);
gtf.apply_normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], train=True, val=True, test=True);

# Set Dataset
gtf.Dataset();

In [ ]:
classes = gtf.system_dict["dataset"]["params"]["classes"];
classes

## Create network by carefully visalizing and arranging

In [ ]:
network = [];
network.append(gtf.convolution(output_channels=64));
network.append(gtf.batch_normalization());
network.append(gtf.relu());
network.append(gtf.convolution(output_channels=64));
network.append(gtf.batch_normalization());
network.append(gtf.relu());
network.append(gtf.max_pooling());

In [ ]:
gtf.debug_custom_model_design(network);

In [ ]:
subnetwork = [];
branch1 = [];
branch1.append(gtf.convolution(output_channels=64));
branch1.append(gtf.batch_normalization());
branch1.append(gtf.convolution(output_channels=64));
branch1.append(gtf.batch_normalization());

branch2 = [];
branch2.append(gtf.convolution(output_channels=64));
branch2.append(gtf.batch_normalization());

branch3 = [];
branch3.append(gtf.identity())

subnetwork.append(branch1);
subnetwork.append(branch2);
subnetwork.append(branch3);
subnetwork.append(gtf.concatenate());

network.append(subnetwork);

In [ ]:
gtf.debug_custom_model_design(network);

In [ ]:
network.append(gtf.convolution(output_channels=128));
network.append(gtf.batch_normalization());
network.append(gtf.relu());
network.append(gtf.max_pooling());

In [ ]:
gtf.debug_custom_model_design(network);

In [ ]:
subnetwork = [];
branch1 = [];
branch1.append(gtf.convolution(output_channels=128));
branch1.append(gtf.batch_normalization());
branch1.append(gtf.convolution(output_channels=128));
branch1.append(gtf.batch_normalization());

branch2 = [];
branch2.append(gtf.convolution(output_channels=128));
branch2.append(gtf.batch_normalization());

branch3 = [];
branch3.append(gtf.identity())

subnetwork.append(branch1);
subnetwork.append(branch2);
subnetwork.append(branch3);
subnetwork.append(gtf.add());

network.append(subnetwork);

In [ ]:
gtf.debug_custom_model_design(network);

In [ ]:
network.append(gtf.convolution(output_channels=256));
network.append(gtf.batch_normalization());
network.append(gtf.relu());
network.append(gtf.max_pooling());

In [ ]:
gtf.debug_custom_model_design(network);

In [ ]:
network.append(gtf.flatten());
network.append(gtf.fully_connected(units=1024));
network.append(gtf.dropout(drop_probability=0.2));
network.append(gtf.fully_connected(units=len(classes)));

In [ ]:
gtf.Compile_Network(network, data_shape=(3, 28, 28));

In [ ]:
gtf.Training_Params(num_epochs=10, display_progress=True, display_progress_realtime=True, 
        save_intermediate_models=False, intermediate_model_prefix="intermediate_model_", save_training_logs=True);

gtf.optimizer_sgd(0.01);
gtf.lr_fixed();
gtf.loss_softmax_crossentropy();

In [ ]:
gtf.Train();

In [ ]:
inference_dataset = "images/test/";
output = gtf.Infer(img_dir=inference_dataset);

In [ ]:
# Create submission
import pandas as pd
sub = pd.read_csv("/kaggle/input/Kannada-MNIST/sample_submission.csv");
for i in range(len(output)):
    index = int(sub[sub['id']==int(output[i]['img_name'].split(".")[0])].index[0])
    #print(output[i]);
    #print(index);
    sub['label'][index] = int(output[i]['predicted_class'])
    #print(sub.iloc[index:index+1])
    #break
sub.to_csv("submission.csv", index=False);

In [ ]:
sub[:10]

In [ ]:
!rm -r images